In [73]:
import requests
import json
import pandas as pd
from matplotlib import pyplot as plt
import os
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.express as px
key = 'MD9WC-r9IQ61zOa3fuqKSbw4TT94t4-pMjUC'

In [32]:
csv_paths = [file for file in os.listdir() if file.endswith("csv")]

In [7]:
# csv_paths = ["df_71.csv", "df_airport.csv"]

In [8]:
def load_clean(path):
    df = pd.read_csv(path)
    df['start_time'] = pd.to_datetime(df['start_time'], utc=True)
    return df

In [9]:
def load_clean_all(csv_paths):
    return {path.strip(".csv"): load_clean(path) for path in csv_paths}

In [10]:
dfs = load_clean_all(csv_paths)

<ipython-input-10-cd484dd33bef>:1: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



In [11]:
def time_sorter(dfs_dict):
    df_hourly = pd.DataFrame()   
    for k in dfs_dict:
        v = dfs_dict[k]
        x = v.groupby(v['start_time'].dt.hour)['record_id'].count()
        df_hourly[f'{k}'] = x
    return df_hourly

In [12]:
hour_traffic = time_sorter(dfs)

In [13]:
hour_traffic.columns

Index(['df_360', 'df_71', 'df_lamar', 'df_35', 'df_290', 'df_183',
       'df_airport', 'df_2222', 'df_burnet', 'df_loop1'],
      dtype='object')

In [14]:
# # x = v.groupby(v['start_time'].dt.hour)['record_id'].count()
# z4 = dfs['df_71'].groupby([dfs['df_71']['day_of_week'], dfs['df_71']['start_time'].dt.hour])['record_id'].count()
# ## Hourly traffic (rows) by Day of week (columns)
# z4 = pd.DataFrame(z4).unstack(level = 0)
# z4.columns = z4.columns.droplevel(level = 0)

In [18]:
hour_t_df = hour_traffic.T

In [79]:
hour_traffic.to_csv("hour_traffic.csv")

In [43]:
origin = widgets.Dropdown(options =  
            list(hour_traffic.columns),
#             value = hour_traffic['df_360'],
            description = 'Major Road/Highway',
            disabled = False)
indx = hour_traffic.index            
trace1 = go.bar(x = hour_traffic['df_360'], y = indx, name = 'Hourly Traffic')
g = go.FigureWidget(data = trace1, layout = go.Layou(title = dict(text = 'Traffic Database')))
g.show()

TypeError: 'module' object is not callable

In [78]:
x1 = [i for i in range(0, 23)]
y1 = [hour for hour in hour_traffic.index]
# plt1 = go.bar(hour_traffic, x = x1, y = y1)
plt1 = px.bar(hour_traffic, x = hour_traffic['df_360'], y = hour_traffic.index)
plt1.show()

In [ ]:
def response(change):
    if origin.value in hour_traffic.columns:
        temp_df = hour_traffic[origin.value] 
        

In [29]:
hour_traffic.iloc(hour_traffic.columns == 'df_360')

TypeError: unhashable type: 'numpy.ndarray'